In [ ]:
import pandas as pd
import numpy as np
import utilities

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
LINE = 'TT7'

In [ ]:
file = f'/data22/bde/Data/NYU-METS/Multivariate_Dataset/dataset/{LINE}/{LINE}.txt'

df = pd.read_csv(file, names=['bandwidth', 'LTE-neighbors', 'RSSI', 'RSRQ', 'ENodeB-change', 'time-advance', 'speed', 'band'])
#df = pd.read_csv(file, header=None)
#df.isnull().values.any()
df.info()

In [ ]:
df['bandwidth'] = pd.to_numeric(df['bandwidth'], downcast='float')
df['LTE-neighbors'] = pd.to_numeric(df['LTE-neighbors'], downcast='float')
df['RSSI'] = pd.to_numeric(df['RSSI'], downcast='float')
df['RSRQ'] = pd.to_numeric(df['RSRQ'], downcast='float')
df['ENodeB-change'] = pd.to_numeric(df['ENodeB-change'], downcast='float')
df['time-advance'] = pd.to_numeric(df['time-advance'], downcast='float')
df['speed'] = pd.to_numeric(df['speed'], downcast='float')
df['band'] = pd.to_numeric(df['band'], downcast='float')

df = utilities.reduce_mem_usage_32(df)

df['date'] = pd.date_range(start='1/1/2023', periods=len(df), freq='S')
df.head()

df.info()
#df.info(show_counts=True)
#df.head()
#df.plot(subplots=True, layout=(3,3), figsize=(40, 25))
#df.to_csv('5Gdataset-{}.csv'.format(file), encoding='utf-8', index=False)

In [ ]:
df.head()

In [ ]:
df.to_csv(f'data/NYU-METS/{LINE}.csv', encoding='utf-8', index=False)